To build a travel recommendation system with the provided datasets, we can approach it in various ways, focusing on content-based filtering, collaborative filtering, and hybrid models. Below is a step-by-step guide with code to implement these models in a Jupyter Notebook:

# Step 1: Import Libraries and Load Data

We'll start by importing necessary libraries and loading the provided datasets into Pandas DataFrames.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns



In [5]:

destinations_df = pd.read_csv("data/destinations.csv")
reviews_df = pd.read_csv("data/reviews.csv")
userhistory_df = pd.read_csv("data/user_history.csv")
users_df = pd.read_csv("data/users.csv")

In [24]:
print("Columns of:")

print(f"Destination : {destinations_df.columns}\n")
print(f"Reviews : {reviews_df.columns}\n")
print(f"User History : {userhistory_df.columns}\n")
print(f"Users : {users_df.columns}\n")


Columns of:
Destination : Index(['DestinationID', 'Name', 'State', 'Type', 'Popularity',
       'BestTimeToVisit'],
      dtype='object')

Reviews : Index(['ReviewID', 'DestinationID', 'UserID', 'Rating', 'ReviewText'], dtype='object')

User History : Index(['HistoryID', 'UserID', 'DestinationID', 'VisitDate',
       'ExperienceRating'],
      dtype='object')

Users : Index(['UserID', 'Name', 'Email', 'Preferences', 'Gender', 'NumberOfAdults',
       'NumberOfChildren'],
      dtype='object')



In [6]:
destinations_df.head()

,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
0,1,Taj Mahal,Uttar Pradesh,Historical,8.691906,Nov-Feb
1,2,Goa Beaches,Goa,Beach,8.605032,Nov-Mar
2,3,Jaipur City,Rajasthan,City,9.225372,Oct-Mar
3,4,Kerala Backwaters,Kerala,Nature,7.977386,Sep-Mar
4,5,Leh Ladakh,Jammu and Kashmir,Adventure,8.399822,Apr-Jun


In [7]:
reviews_df.head()

,ReviewID,DestinationID,UserID,Rating,ReviewText
0,1,178,327,2,Incredible monument!
1,2,411,783,1,Loved the beaches!
2,3,927,12,2,A historical wonder
3,4,358,959,3,Incredible monument!
4,5,989,353,2,Loved the beaches!


In [8]:
userhistory_df.head()

,HistoryID,UserID,DestinationID,VisitDate,ExperienceRating
0,1,525,760,2024-01-01,3
1,2,184,532,2024-02-15,5
2,3,897,786,2024-03-20,2
3,4,470,660,2024-01-01,1
4,5,989,389,2024-02-15,4


In [9]:
users_df.head()

,UserID,Name,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,1,Kavya,kavya@example.com,"Beaches, Historical",Female,1,0
1,2,Rohan,rohan@example.com,"Nature, Adventure",Male,2,2
2,3,Kavya,kavya@example.com,"City, Historical",Female,2,0
3,4,Anika,anika@example.com,"Beaches, Historical",Female,1,0
4,5,Tanvi,tanvi@example.com,"Nature, Adventure",Female,2,2


In [10]:
# Merge datasets step-by-step
# Step 1: Merge reviews with destinations on 'DestinationID'
reviews_destinations = pd.merge(reviews_df, destinations_df, on='DestinationID', how='inner')

# Step 2: Merge the result with user history on 'UserID'
reviews_destinations_userhistory = pd.merge(reviews_destinations, userhistory_df, on='UserID', how='inner')

# Step 3: Merge the result with users on 'UserID'
df = pd.merge(reviews_destinations_userhistory, users_df, on='UserID', how='inner')

df

,ReviewID,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,1,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,2,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,4,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,5,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,6,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


# Step 2: Data Preprocessing 

We will clean and preprocess the data to make it ready for building recommendation models.

In [11]:
df.drop_duplicates(inplace=True)
df.shape

(993, 20)

In [12]:
df.to_csv("final_df.csv",index=False)

In [25]:
final_df = pd.read_csv('final_df.csv')

In [26]:
final_df.columns

Index(['ReviewID', 'DestinationID_x', 'UserID', 'Rating', 'ReviewText',
       'Name_x', 'State', 'Type', 'Popularity', 'BestTimeToVisit', 'HistoryID',
       'DestinationID_y', 'VisitDate', 'ExperienceRating', 'Name_y', 'Email',
       'Preferences', 'Gender', 'NumberOfAdults', 'NumberOfChildren'],
      dtype='object')

# Step 3: Content-Based Recommendation Model

Content-based recommendations recommend destinations based on the attributes of the destination that the user has shown interest in (i.e., based on the "Preferences" and "Type" columns).

### Step 3.1: Prepare the Destination Features
We'll use the "Type", "State", and "BestTimeToVisit" columns to create features for content-based filtering.

In [40]:
destinations_df['features'] = (
    destinations_df['State'] + ' ' + destinations_df['Type'] + ' ' +
    destinations_df['Popularity'].astype(str) + ' ' + destinations_df['BestTimeToVisit']
)

# Convert text data into numerical vectors
vectorizer = TfidfVectorizer(stop_words='english')
destination_features = vectorizer.fit_transform(destinations_df['features'])

# Compute similarity matrix
cosine_sim = cosine_similarity(destination_features, destination_features)



In [41]:

def recommend_destinations_by_name(destination_name, top_n=5):
    """
    Recommends top N unique destinations based on content similarity.
    
    Args:
    - destination_name: Name of the destination to get recommendations for.
    - top_n: Number of recommendations to return (default: 5).
    
    Returns:
    - DataFrame with recommended destinations.
    """
    try:
        idx = destinations_df[destinations_df['Name'].str.lower() == destination_name.lower()].index[0]
    except IndexError:
        return pd.DataFrame({"Error": [f"Destination '{destination_name}' not found in the dataset."]})
    
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    recommendations = []
    seen_destinations = set()
    
    for i, _ in similarity_scores:
        destination = destinations_df.iloc[i]
        if destination['Name'] != destination_name and destination['Name'] not in seen_destinations:
            recommendations.append({
                'Name': destination['Name'],
                'State': destination['State'],
                'Type': destination['Type'],
                'Popularity': destination['Popularity'],
                'BestTimeToVisit': destination['BestTimeToVisit']
            })
            seen_destinations.add(destination['Name'])
        if len(recommendations) >= top_n:
            break
    
    return pd.DataFrame(recommendations)

In [42]:
print(recommend_destinations_by_name("Taj Mahal"))

                Name              State       Type  Popularity BestTimeToVisit
0        Goa Beaches                Goa      Beach    8.605032         Nov-Mar
1        Jaipur City          Rajasthan       City    9.225372         Oct-Mar
2  Kerala Backwaters             Kerala     Nature    7.977386         Sep-Mar
3         Leh Ladakh  Jammu and Kashmir  Adventure    8.399822         Apr-Jun


### Step 3.2: Recommendation Function 

Based on the cosine similarity matrix, we can recommend destinations similar to the ones the user has liked in the past.

In [47]:
# Define recommendation function
def recommend_destinations(destination_name, top_n=5):
    """
    Recommends top N unique destinations based on content similarity and user history.

    Args:
    - destination_name: Name of the destination to get recommendations for.
    - top_n: Number of recommendations to return (default: 5).

    Returns:
    - DataFrame with recommended destinations.
    """
    try:
        idx = destinations_df[destinations_df['Name'].str.lower() == destination_name.lower()].index[0]
    except IndexError:
        return pd.DataFrame({"Error": [f"Destination '{destination_name}' not found in the dataset."]})

    # Content-based recommendations
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get users who visited the given destination
    visited_users = userhistory_df[userhistory_df['DestinationID'] == destinations_df.iloc[idx]['DestinationID']]['UserID'].unique()

    # Find other destinations these users visited
    user_based_destinations = userhistory_df[userhistory_df['UserID'].isin(visited_users)]['DestinationID'].unique()

    recommendations = []
    seen_destinations = set()

    # First, add user-based recommendations
    for dest_id in user_based_destinations:
        destination = destinations_df[destinations_df['DestinationID'] == dest_id].iloc[0]
        if destination['Name'] != destination_name and destination['Name'] not in seen_destinations:
            recommendations.append({
                'Name': destination['Name'],
                'State': destination['State'],
                'Type': destination['Type'],
                'Popularity': destination['Popularity'],
                'BestTimeToVisit': destination['BestTimeToVisit'],
                'Source': 'User-Based'
            })
            seen_destinations.add(destination['Name'])
        if len(recommendations) >= top_n:
            break  # Ensure we don't exceed the top_n limit

    # If we still need more recommendations, add content-based ones
    if len(recommendations) < top_n:
        for i, _ in similarity_scores:
            destination = destinations_df.iloc[i]
            if destination['Name'] != destination_name and destination['Name'] not in seen_destinations:
                recommendations.append({
                    'Name': destination['Name'],
                    'State': destination['State'],
                    'Type': destination['Type'],
                    'Popularity': destination['Popularity'],
                    'BestTimeToVisit': destination['BestTimeToVisit']
                })
                seen_destinations.add(destination['Name'])
            if len(recommendations) >= top_n:
                break

    return pd.DataFrame(recommendations)

# Example: Recommend destinations for user with ID 1
recommended_destinations = recommend_destinations("Taj Mahal")

# Display recommendations
recommended_destinations

,Name,State,Type,Popularity,BestTimeToVisit
0,Goa Beaches,Goa,Beach,8.605032,Nov-Mar
1,Jaipur City,Rajasthan,City,9.225372,Oct-Mar
2,Kerala Backwaters,Kerala,Nature,7.977386,Sep-Mar
3,Leh Ladakh,Jammu and Kashmir,Adventure,8.399822,Apr-Jun


In [49]:
# Example: Recommend destinations for user with ID 1
recommended_destinations = recommend_destinations("Kerala")

# Display recommendations
recommended_destinations

,Error
0,Destination 'Kerala' not found in the dataset.


In [22]:
# Example: Recommend destinations for user with ID 1
recommended_destinations = recommend_destinations(98, userhistory_df, destinations_df, cosine_sim)

# Display recommendations
recommended_destinations

,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
0,338,Jaipur City,Rajasthan,City,8.139655,Oct-Mar
1,921,Taj Mahal,Uttar Pradesh,Historical,8.685685,Nov-Feb
2,329,Kerala Backwaters,Kerala,Nature,8.214898,Sep-Mar
3,336,Taj Mahal,Uttar Pradesh,Historical,9.268792,Nov-Feb
4,337,Goa Beaches,Goa,Beach,8.576363,Nov-Mar


# Step 4: Collaborative Filtering
In collaborative filtering, we recommend destinations based on the behavior and preferences of other similar users.

#### Step 4.1: Create a User-Item Matrix
We'll create a matrix where rows represent users, and columns represent destinations. The values will represent the ratings or experience of the user for each destination.

In [50]:
# Create a user-item matrix based on user history
user_item_matrix = userhistory_df.pivot(index='UserID', columns='DestinationID', values='ExperienceRating')

# Fill missing values with 0 (indicating no rating/experience)
user_item_matrix.fillna(0, inplace=True)

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

### Step 4.2: Cosine Similarity Between Users
We will compute the cosine similarity between users based on the user-item matrix

In [51]:
# Function to recommend destinations based on user similarity
def collaborative_recommend(user_id, user_similarity, user_item_matrix, destinations_df):
    """
    Recommends destinations based on collaborative filtering.

    Args:
    - user_id: ID of the user for whom recommendations are to be made.
    - user_similarity: Cosine similarity matrix for users.
    - user_item_matrix: User-item interaction matrix (e.g., ratings or preferences).
    - destinations_df: DataFrame containing destination details.

    Returns:
    - DataFrame with recommended destinations and their details.
    """
    # Find similar users
    similar_users = user_similarity[user_id - 1]
    
    # Get the top 5 most similar users
    similar_users_idx = np.argsort(similar_users)[::-1][1:6]
    
    # Get the destinations liked by similar users
    similar_user_ratings = user_item_matrix.iloc[similar_users_idx].mean(axis=0)
    
    # Recommend the top 5 destinations
    recommended_destinations_ids = similar_user_ratings.sort_values(ascending=False).head(5).index
    
    # Filter the destinations DataFrame to include detailed information
    recommendations = destinations_df[destinations_df['DestinationID'].isin(recommended_destinations_ids)][[
        'DestinationID', 'Name', 'State', 'Type', 'Popularity', 'BestTimeToVisit'
    ]]
    
    return recommendations

# Example: Collaborative recommendations for user 1
collaborative_recommendations = collaborative_recommend(1, user_similarity, user_item_matrix, destinations_df)

# Display recommendations
collaborative_recommendations


,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
335,336,Taj Mahal,Uttar Pradesh,Historical,9.268792,Nov-Feb
519,520,Leh Ladakh,Jammu and Kashmir,Adventure,8.521876,Apr-Jun
684,685,Leh Ladakh,Jammu and Kashmir,Adventure,7.598827,Apr-Jun
784,785,Leh Ladakh,Jammu and Kashmir,Adventure,9.366933,Apr-Jun
878,879,Kerala Backwaters,Kerala,Nature,8.309631,Sep-Mar


In [52]:
# Example: Collaborative recommendations for user 1
collaborative_recommendations = collaborative_recommend(11, user_similarity, user_item_matrix, destinations_df)

# Display recommendations
collaborative_recommendations

,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
5,6,Taj Mahal,Uttar Pradesh,Historical,7.648950,Nov-Feb
367,368,Jaipur City,Rajasthan,City,8.008252,Oct-Mar
519,520,Leh Ladakh,Jammu and Kashmir,Adventure,8.521876,Apr-Jun
784,785,Leh Ladakh,Jammu and Kashmir,Adventure,9.366933,Apr-Jun
969,970,Leh Ladakh,Jammu and Kashmir,Adventure,9.013101,Apr-Jun


In [53]:
# Example: Collaborative recommendations for user 1
collaborative_recommendations = collaborative_recommend(100, user_similarity, user_item_matrix, destinations_df)

# Display recommendations
collaborative_recommendations

,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
5,6,Taj Mahal,Uttar Pradesh,Historical,7.648950,Nov-Feb
346,347,Goa Beaches,Goa,Beach,7.920960,Nov-Mar
519,520,Leh Ladakh,Jammu and Kashmir,Adventure,8.521876,Apr-Jun
684,685,Leh Ladakh,Jammu and Kashmir,Adventure,7.598827,Apr-Jun
784,785,Leh Ladakh,Jammu and Kashmir,Adventure,9.366933,Apr-Jun


# Step 6 User input feature base recommendation

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # Using regressor for continuous target
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

# Load and preprocess the data
data = pd.read_csv('final_df.csv')

In [58]:
final_df.columns

Index(['ReviewID', 'DestinationID_x', 'UserID', 'Rating', 'ReviewText',
       'Name_x', 'State', 'Type', 'Popularity', 'BestTimeToVisit', 'HistoryID',
       'DestinationID_y', 'VisitDate', 'ExperienceRating', 'Name_y', 'Email',
       'Preferences', 'Gender', 'NumberOfAdults', 'NumberOfChildren'],
      dtype='object')

In [55]:
# Select relevant features
features = ['Name_x', 'State', 'Type', 'BestTimeToVisit', 'Preferences', 'Gender', 'NumberOfAdults', 'NumberOfChildren']
target = 'Popularity'  # Predicting popularity

# Encode categorical variables
label_encoders = {}
for col in features:
    if data[col].dtype == 'object':
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

# Split the dataset
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")


Mean Squared Error: 0.30
R² Score: 0.07


In [56]:
import pandas as pd

# Prediction system
def recommend_destinations(user_input, model, label_encoders, features, data):
    # Encode user input
    encoded_input = {}
    for feature in features:
        if feature in label_encoders:
            encoded_input[feature] = label_encoders[feature].transform([user_input[feature]])[0]
        else:
            encoded_input[feature] = user_input[feature]

    # Convert to DataFrame
    input_df = pd.DataFrame([encoded_input])

    # Predict popularity
    predicted_popularity = model.predict(input_df)[0]

    return predicted_popularity


# Example user input
user_input = {
    'Name_x': 'Jaipur City',
    'Type': 'City',
    'State': 'Rajasthan',
    'BestTimeToVisit': 'Oct-Mar',  # This might be an unseen label
    'Preferences': 'City, Historical',
    'Gender': 'Female',
    'NumberOfAdults': 2,
    'NumberOfChildren': 1,
}

# Make a prediction
predicted_popularity = recommend_destinations(user_input, model, label_encoders, features, data)

# Print the predicted popularity and the recommended destination details
print(f"Predicted Popularity Score: {predicted_popularity:.2f}")

Predicted Popularity Score: 7.88


In [57]:
import pickle

pickle.dump(model, open("model.pkl",'wb'))
pickle.dump(label_encoders, open("label_encoders.pkl",'wb'))